# 5.8 - Redes Neurais Artificiais

In [37]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score,precision_score,recall_score
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from IPython.display import display
from sklearn.preprocessing import scale
from keras.wrappers.scikit_learn import KerasClassifier

import seaborn as sns

from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from time import time
from keras.utils import np_utils

os.chdir('D:\\Users\\Diogo\\Documents\\Pós Graduação\\Módulos\\TCC\\DataSets\\Futebol Brasileiro')

In [91]:
#Carregando os dados do Data Frame

df_dados=pd.read_csv('dados_normalizados.csv',sep=',')

df_dados.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ID,campeonato,rodata,data,clube_mandante,clube_visitante,qtd_cartao_amarelo_mandante,...,FalAma_mandante,FalVer_mandante,FalAma_visitante,FalVer_visitante,FalAmaTemp_mandante,FalVerTemp_mandante,FalAmaTemp_visitante,FalVerTemp_visitante,Target_mandante_amarelo,Target_visitante_amarelo
0,0,0,0,4987,2015,1,9/5/2015,Chapecoense,Coritiba,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1,1,4988,2015,1,9/5/2015,Palmeiras,Atletico-MG,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,2,2,2,4989,2015,1,9/5/2015,Fluminense,Joinville,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,3,3,3,4990,2015,1,10/5/2015,Gremio,Ponte Preta,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,4,4,4,4991,2015,1,10/5/2015,Athletico-PR,Internacional,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [92]:
df_dados_norm = df_dados.copy(deep=True)

In [93]:
#Selecionando as colunas que serão utilizadas no modelo

df_dados_norm_visitante=df_dados_norm[['classico_regional','MedAma5_mandante','MedAma5_visitante',
'MedAmaTemp_mandante','MedAmaTemp_visitante','MedAmaH_mandante',
'MedAmaH_visitante','MaxAmaTemp_mandante','MinAmaTemp_mandante',
'MaxAmaTemp_visitante','MinAmaTemp_visitante','ScoreMed_mandante','ScoreLevel_mandante','ScoreMedTemp_mandante',
                             'ScoreLevelTemp_mandante','FalAma_mandante','FalAmaTemp_mandante','FalAma_visitante',
                             'FalAmaTemp_visitante','ScoreMed_visitante','ScoreLevel_visitante','ScoreLevelTemp_mandante',
                            'ScoreMedTemp_visitante']]

target= df_dados_norm[['Target_visitante_amarelo']]
y=target.copy()
X=df_dados_norm_visitante.copy()

In [94]:
#Dividindo os Dataframes que serão utilzados no modelo

X_train=df_dados_norm_visitante[:2128]
X_test=df_dados_norm_visitante[2128:3040]
y_train=target[:2128]
y_test=target[2128:3040]

In [39]:
#Código de para aplicar o Grid Search

def create_model(neurons, optimizer):
    model = Sequential()
    model.add(Dense(neurons, input_dim=23, activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(1)) 
    model.add(Activation('sigmoid'))  
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,             
                  metrics=['accuracy'])
    return model
    


# Definindo a faixa de parâmetros
param_grid = {'neurons': [2, 8, 16],
              'batch_size': [4, 16],
              'optimizer': ['SGD', 'RMSprop', 'Adam']} 



#Defina o modelo usando o método KerasClassifier.
#Isso torna nosso modelo Keras disponível para GridSearch
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=200, verbose=1)
 
#Fornecendo a métrica para a validação cruzada KFold. cv=3 é um bom ponto de partida
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)

#A execução demora bastante com base no número de parâmetros e divisões de cv 
#No nosso caso - 18 * 3 * 2 * num_epochs = 21600 épocas totais se epochs=200
grid_result = grid.fit(X_train, y_train)


print("Best accuracy of: %f using %s" % (grid_result.best_score_, 
                                         grid_result.best_params_))



Epoch 1/200
355/355 [==============================] - 1s 907us/step - loss: 0.7041 - accuracy: 0.5113
Epoch 2/200
355/355 [==============================] - 0s 1ms/step - loss: 0.6921 - accuracy: 0.5543
Epoch 3/200
355/355 [==============================] - 0s 1ms/step - loss: 0.6897 - accuracy: 0.5543
Epoch 4/200
355/355 [==============================] - 0s 901us/step - loss: 0.6878 - accuracy: 0.5543
Epoch 5/200
355/355 [==============================] - 0s 895us/step - loss: 0.6878 - accuracy: 0.5543
Epoch 6/200
355/355 [==============================] - 0s 907us/step - loss: 0.6879 - accuracy: 0.5543
Epoch 7/200
355/355 [==============================] - 0s 887us/step - loss: 0.6878 - accuracy: 0.5543
Epoch 8/200
355/355 [==============================] - 0s 887us/step - loss: 0.6879 - accuracy: 0.5543
Epoch 9/200
355/355 [==============================] - 0s 890us/step - loss: 0.6878 - accuracy: 0.5543
Epoch 10/200
355/355 [==============================] - 0s 893us/step - loss:

355/355 [==============================] - 0s 870us/step - loss: 0.6713 - accuracy: 0.5543
Epoch 158/200
355/355 [==============================] - 0s 864us/step - loss: 0.6706 - accuracy: 0.5529
Epoch 159/200
355/355 [==============================] - 0s 853us/step - loss: 0.6682 - accuracy: 0.5543
Epoch 160/200
355/355 [==============================] - 0s 847us/step - loss: 0.6656 - accuracy: 0.5543
Epoch 161/200
355/355 [==============================] - 0s 864us/step - loss: 0.6653 - accuracy: 0.5543
Epoch 162/200
355/355 [==============================] - 0s 881us/step - loss: 0.6683 - accuracy: 0.5543
Epoch 163/200
355/355 [==============================] - 0s 938us/step - loss: 0.6702 - accuracy: 0.5543
Epoch 164/200
355/355 [==============================] - 0s 904us/step - loss: 0.6687 - accuracy: 0.5543
Epoch 165/200
355/355 [==============================] - 0s 884us/step - loss: 0.6663 - accuracy: 0.5543
Epoch 166/200
355/355 [==============================] - 0s 864us/ste

355/355 [==============================] - 0s 884us/step - loss: 0.6833 - accuracy: 0.5447
Epoch 113/200
355/355 [==============================] - 0s 853us/step - loss: 0.6824 - accuracy: 0.5476
Epoch 114/200
355/355 [==============================] - 0s 904us/step - loss: 0.6835 - accuracy: 0.5476
Epoch 115/200
355/355 [==============================] - 0s 895us/step - loss: 0.6817 - accuracy: 0.5518
Epoch 116/200
355/355 [==============================] - 0s 850us/step - loss: 0.6832 - accuracy: 0.5476
Epoch 117/200
355/355 [==============================] - 0s 862us/step - loss: 0.6791 - accuracy: 0.5525
Epoch 118/200
355/355 [==============================] - 0s 876us/step - loss: 0.6826 - accuracy: 0.5469
Epoch 119/200
355/355 [==============================] - 0s 862us/step - loss: 0.6838 - accuracy: 0.5511
Epoch 120/200
355/355 [==============================] - 0s 864us/step - loss: 0.6844 - accuracy: 0.5476
Epoch 121/200
355/355 [==============================] - 0s 850us/ste

Epoch 190/200
355/355 [==============================] - 0s 884us/step - loss: 0.6780 - accuracy: 0.5574
Epoch 191/200
355/355 [==============================] - 0s 853us/step - loss: 0.6764 - accuracy: 0.5624
Epoch 192/200
355/355 [==============================] - 0s 847us/step - loss: 0.6758 - accuracy: 0.5539
Epoch 193/200
355/355 [==============================] - 0s 938us/step - loss: 0.6735 - accuracy: 0.5539
Epoch 194/200
355/355 [==============================] - 0s 912us/step - loss: 0.6740 - accuracy: 0.5666
Epoch 195/200
355/355 [==============================] - 0s 904us/step - loss: 0.6772 - accuracy: 0.5546
Epoch 196/200
355/355 [==============================] - 0s 887us/step - loss: 0.6769 - accuracy: 0.5539
Epoch 197/200
355/355 [==============================] - 0s 867us/step - loss: 0.6757 - accuracy: 0.5567
Epoch 198/200
355/355 [==============================] - 0s 864us/step - loss: 0.6756 - accuracy: 0.5624
Epoch 199/200
355/355 [==============================] 

355/355 [==============================] - 0s 870us/step - loss: 0.6846 - accuracy: 0.5398
Epoch 146/200
355/355 [==============================] - 0s 873us/step - loss: 0.6831 - accuracy: 0.5349
Epoch 147/200
355/355 [==============================] - 0s 862us/step - loss: 0.6848 - accuracy: 0.5321
Epoch 148/200
355/355 [==============================] - 0s 864us/step - loss: 0.6853 - accuracy: 0.5370
Epoch 149/200
355/355 [==============================] - 0s 856us/step - loss: 0.6845 - accuracy: 0.5328
Epoch 150/200
355/355 [==============================] - 0s 850us/step - loss: 0.6826 - accuracy: 0.5250
Epoch 151/200
355/355 [==============================] - 0s 876us/step - loss: 0.6846 - accuracy: 0.5215
Epoch 152/200
355/355 [==============================] - 0s 958us/step - loss: 0.6876 - accuracy: 0.5173
Epoch 153/200
355/355 [==============================] - 0s 856us/step - loss: 0.6846 - accuracy: 0.5271
Epoch 154/200
355/355 [==============================] - 0s 856us/ste

...

89/89 [==============================] - 0s 1ms/step - loss: 0.6760 - accuracy: 0.5603
Epoch 21/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6763 - accuracy: 0.5546
Epoch 22/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6811 - accuracy: 0.5504
Epoch 23/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6803 - accuracy: 0.5743
Epoch 24/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6788 - accuracy: 0.5539
Epoch 25/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6786 - accuracy: 0.5715
Epoch 26/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6729 - accuracy: 0.5758
Epoch 27/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6809 - accuracy: 0.5631
Epoch 28/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6761 - accuracy: 0.5751
Epoch 29/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6729 - accuracy: 0.5906
Epoch 30/200


89/89 [==============================] - 0s 1ms/step - loss: 0.6306 - accuracy: 0.6540
Epoch 183/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6342 - accuracy: 0.6364
Epoch 184/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6335 - accuracy: 0.6237
Epoch 185/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6380 - accuracy: 0.6293
Epoch 186/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6381 - accuracy: 0.6385
Epoch 187/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6306 - accuracy: 0.6413
Epoch 188/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6282 - accuracy: 0.6505
Epoch 189/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6390 - accuracy: 0.6328
Epoch 190/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6315 - accuracy: 0.6321
Epoch 191/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6269 - accuracy: 0.6469
Epoc

89/89 [==============================] - 0s 1ms/step - loss: 0.6645 - accuracy: 0.5962
Epoch 64/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6676 - accuracy: 0.5856
Epoch 65/200
89/89 [==============================] - 0s 1000us/step - loss: 0.6647 - accuracy: 0.5891
Epoch 66/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6665 - accuracy: 0.5877
Epoch 67/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6612 - accuracy: 0.6032
Epoch 68/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6649 - accuracy: 0.5962
Epoch 69/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6618 - accuracy: 0.5990
Epoch 70/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6607 - accuracy: 0.5941
Epoch 71/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6620 - accuracy: 0.5821
Epoch 72/200
89/89 [==============================] - 0s 1ms/step - loss: 0.6584 - accuracy: 0.5990
Epoch 73/2

133/133 [==============================] - 0s 939us/step - loss: 0.6704 - accuracy: 0.5836
Epoch 24/200
133/133 [==============================] - 0s 909us/step - loss: 0.6731 - accuracy: 0.5667
Epoch 25/200
133/133 [==============================] - 0s 939us/step - loss: 0.6722 - accuracy: 0.5860
Epoch 26/200
133/133 [==============================] - 0s 909us/step - loss: 0.6727 - accuracy: 0.5846
Epoch 27/200
133/133 [==============================] - 0s 924us/step - loss: 0.6720 - accuracy: 0.5761
Epoch 28/200
133/133 [==============================] - 0s 909us/step - loss: 0.6693 - accuracy: 0.5851
Epoch 29/200
133/133 [==============================] - 0s 917us/step - loss: 0.6684 - accuracy: 0.5827
Epoch 30/200
133/133 [==============================] - 0s 917us/step - loss: 0.6686 - accuracy: 0.5860
Epoch 31/200
133/133 [==============================] - 0s 909us/step - loss: 0.6721 - accuracy: 0.5695
Epoch 32/200
133/133 [==============================] - 0s 924us/step - loss:

133/133 [==============================] - 0s 947us/step - loss: 0.6458 - accuracy: 0.6133
Epoch 179/200
133/133 [==============================] - 0s 924us/step - loss: 0.6366 - accuracy: 0.6250
Epoch 180/200
133/133 [==============================] - 0s 909us/step - loss: 0.6409 - accuracy: 0.6194
Epoch 181/200
133/133 [==============================] - 0s 932us/step - loss: 0.6413 - accuracy: 0.6212
Epoch 182/200
133/133 [==============================] - 0s 894us/step - loss: 0.6404 - accuracy: 0.6165
Epoch 183/200
133/133 [==============================] - 0s 939us/step - loss: 0.6383 - accuracy: 0.6241
Epoch 184/200
133/133 [==============================] - 0s 894us/step - loss: 0.6389 - accuracy: 0.6222
Epoch 185/200
133/133 [==============================] - 0s 909us/step - loss: 0.6352 - accuracy: 0.6231
Epoch 186/200
133/133 [==============================] - 0s 917us/step - loss: 0.6402 - accuracy: 0.6133
Epoch 187/200
133/133 [==============================] - 0s 894us/ste

In [41]:
#Verificando os resultados do Grid Search

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

###########################################################

#Let us load the best model and predict on our input data
best_model =grid_result.best_estimator_

# Predicting the Test set results
y_pred = best_model.predict(X)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)
print(cm)
#sns.heatmap(cm, annot=True)

0.509875 (0.020446) with: {'batch_size': 4, 'neurons': 2, 'optimizer': 'SGD'}
0.536667 (0.035503) with: {'batch_size': 4, 'neurons': 2, 'optimizer': 'RMSprop'}
0.521627 (0.032082) with: {'batch_size': 4, 'neurons': 2, 'optimizer': 'Adam'}
0.528204 (0.017346) with: {'batch_size': 4, 'neurons': 8, 'optimizer': 'SGD'}
0.529614 (0.019810) with: {'batch_size': 4, 'neurons': 8, 'optimizer': 'RMSprop'}
0.541819 (0.008522) with: {'batch_size': 4, 'neurons': 8, 'optimizer': 'Adam'}
0.532435 (0.021496) with: {'batch_size': 4, 'neurons': 16, 'optimizer': 'SGD'}
0.525391 (0.024309) with: {'batch_size': 4, 'neurons': 16, 'optimizer': 'RMSprop'}
0.511284 (0.008975) with: {'batch_size': 4, 'neurons': 16, 'optimizer': 'Adam'}
0.530092 (0.026698) with: {'batch_size': 16, 'neurons': 2, 'optimizer': 'SGD'}
0.541372 (0.028282) with: {'batch_size': 16, 'neurons': 2, 'optimizer': 'RMSprop'}
0.507994 (0.018802) with: {'batch_size': 16, 'neurons': 2, 'optimizer': 'Adam'}
0.530552 (0.016112) with: {'batch_size

In [95]:
#Código para treinamento do modelo e exibição dos resultados

modelo = Sequential()
modelo.add(Dense(15,activation='relu',kernel_initializer='random_normal',input_dim=len(X.columns)))

modelo.add(Dense(1,activation='sigmoid',kernel_initializer='random_normal'))

modelo.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
modelo.fit(X_train,y_train,batch_size=16,epochs=200,verbose=False)
pred = modelo.predict(X_test)
y_pred = (pred > 0.5).astype(int)

xpred = modelo.predict(X)
x_pred= (xpred > 0.5).astype(int)

RNN = accuracy_score(y_test,y_pred)

print('Acurácia RNN:{}'.format(RNN))

print ('Matriz de Confusão')
print(confusion_matrix(y_test,y_pred))

print('Relatório Completo')
print(classification_report(y,x_pred))
print('Relatório Teste')
print(classification_report(y_test,y_pred))

Acurácia RNN:0.5466520307354555
Matriz de Confusão
[[331 186]
 [227 167]]
Relatório Completo
              precision    recall  f1-score   support

           0       0.61      0.73      0.67      1655
           1       0.58      0.45      0.51      1384

    accuracy                           0.60      3039
   macro avg       0.60      0.59      0.59      3039
weighted avg       0.60      0.60      0.60      3039

Relatório Teste
              precision    recall  f1-score   support

           0       0.59      0.64      0.62       517
           1       0.47      0.42      0.45       394

    accuracy                           0.55       911
   macro avg       0.53      0.53      0.53       911
weighted avg       0.54      0.55      0.54       911



In [101]:
#Inserção dos resultados no Dataframe para análise.

df_resultados_visitantes=pd.read_csv('df_resultados_visitantes.csv',sep=',')
dados = [
    {"tipo": "Visitantes", "modelo": "Redes Neurais", "acuracia": RNN.round(4), "precisao_teste": precision_score(y_test,y_pred,average='macro').round(4), "precisao_geral": precision_score(y,x_pred,average='macro').round(4), "recall_teste": recall_score(y_test,y_pred,average='macro').round(4), "recall_geral": recall_score(y,x_pred,average='macro').round(4)}
]
df_resultados_visitantes = df_resultados_visitantes.append(dados, ignore_index=True)
df_resultados_visitantes

,Unnamed: 0,Unnamed: 0.1,tipo,modelo,acuracia,precisao_teste,precisao_geral,recall_teste,recall_geral
0,0.0,0.0,Visitantes,Regressão Logística,0.5631,0.5330,0.5654,0.5191,0.5332
1,1.0,NaN,Visitantes,SVC,0.5686,0.5621,0.6357,0.5025,0.5108
2,NaN,NaN,Visitantes,Redes Neurais,0.5467,0.5331,0.5989,0.5320,0.5912


In [102]:
df_dados_norm = df_dados.copy(deep=True)

In [103]:
#Selecionando as colunas que serão utilizadas no modelo

df_dados_norm_mandante=df_dados_norm[['classico_regional','MedAma5_mandante','MedAma5_visitante',
'MedAmaTemp_mandante','MedAmaTemp_visitante','MedAmaH_mandante',
'MedAmaH_visitante','MaxAmaTemp_mandante','MinAmaTemp_mandante',
'MaxAmaTemp_visitante','MinAmaTemp_visitante','ScoreMed_mandante','ScoreLevel_mandante','ScoreMedTemp_mandante',
                             'ScoreLevelTemp_mandante','FalAma_mandante','FalAmaTemp_mandante','FalAma_visitante',
                             'FalAmaTemp_visitante','ScoreMed_visitante','ScoreLevel_visitante','ScoreLevelTemp_mandante',
                            'ScoreMedTemp_visitante']]

target= df_dados_norm[['Target_mandante_amarelo']]
y=target.copy()
X=df_dados_norm_mandante.copy()

In [104]:
#Dividindo os Dataframes que serão utilzados no modelo

X_train=df_dados_norm_mandante[:2128]
X_test=df_dados_norm_mandante[2128:3040]
y_train=target[:2128]
y_test=target[2128:3040]

In [107]:
#Código para treinamento do modelo e exibição dos resultados

modelo = Sequential()
modelo.add(Dense(15,activation='relu',kernel_initializer='random_normal',input_dim=len(X.columns)))

modelo.add(Dense(1,activation='sigmoid',kernel_initializer='random_normal'))

modelo.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
modelo.fit(X_train,y_train,batch_size=16,epochs=200,verbose=False)
pred = modelo.predict(X_test)
y_pred = (pred > 0.5).astype(int)

xpred = modelo.predict(X)
x_pred= (xpred > 0.5).astype(int)

RNN = accuracy_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred,average='micro')

print('Acurácia RNN:{}'.format(RNN))
print('F1 Score :{}'.format(f1))
print ('Matriz de Confusão')
print(confusion_matrix(y_test,y_pred))

print('Relatório Completo')
print(classification_report(y,x_pred))
print('Relatório Teste')
print(classification_report(y_test,y_pred))

Acurácia RNN:0.6158068057080132
F1 Score :0.6158068057080132
Matriz de Confusão
[[483  70]
 [280  78]]
Relatório Completo
              precision    recall  f1-score   support

           0       0.66      0.92      0.77      1877
           1       0.65      0.24      0.35      1162

    accuracy                           0.66      3039
   macro avg       0.65      0.58      0.56      3039
weighted avg       0.66      0.66      0.61      3039

Relatório Teste
              precision    recall  f1-score   support

           0       0.63      0.87      0.73       553
           1       0.53      0.22      0.31       358

    accuracy                           0.62       911
   macro avg       0.58      0.55      0.52       911
weighted avg       0.59      0.62      0.57       911



In [108]:
#Inserção dos resultados no Dataframe para análise.

df_resultados_mandantes=pd.read_csv('df_resultados_mandantes.csv',sep=',')
dados = [
    {"tipo": "Mandantes", "modelo": "Redes Neurais", "acuracia": RNN.round(4), "precisao_teste": precision_score(y_test,y_pred,average='macro').round(4), "precisao_geral": precision_score(y,x_pred,average='macro').round(4), "recall_teste": recall_score(y_test,y_pred,average='macro').round(4), "recall_geral": recall_score(y,x_pred,average='macro').round(4)}

    ]
df_resultados_mandantes = df_resultados_mandantes.append(dados, ignore_index=True)
df_resultados_mandantes

,Unnamed: 0,Unnamed: 0.1,tipo,modelo,acuracia,precisao_teste,precisao_geral,recall_teste,recall_geral
0,0.0,0.0,Mandantes,Regressão Logística,0.6103,0.5646,0.5929,0.5384,0.5250
1,1.0,NaN,Mandantes,SVC,0.6037,0.5396,0.6832,0.5081,0.5439
2,NaN,NaN,Mandantes,Redes Neurais,0.6158,0.5800,0.6544,0.5456,0.5796


In [109]:
#Salvando os Dataframes em diretório local

df_resultados_mandantes.to_csv('D:\\Users\\Diogo\\Documents\\Pós Graduação\\Módulos\\TCC\\DataSets\\Futebol Brasileiro\\df_resultados_mandantes.csv')
df_resultados_visitantes.to_csv('D:\\Users\\Diogo\\Documents\\Pós Graduação\\Módulos\\TCC\\DataSets\\Futebol Brasileiro\\df_resultados_visitantes.csv')